In [1]:
!pip install mlxtend

In [2]:
import pandas as pd
import numpy as np

In [3]:
diabetes=pd.read_csv('C:/Users/shubham.chhetri/Desktop/MYPROJECT/Machine learning/Preparing data for ML/diabetes.csv')

In [4]:
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
x=diabetes.drop('Outcome',axis=1)
y=diabetes['Outcome']

In [6]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [7]:
model=LogisticRegression(solver='liblinear')
rfe=RFE(model,n_features_to_select=4)

In [8]:
fit=rfe.fit(x,y)

In [9]:
print("num features:",fit.n_features_)
print("num features:",fit.support_)
print("num features:",fit.ranking_)

num features: 4
num features: [ True  True False False False  True  True False]
num features: [1 1 2 4 5 1 1 3]


In [10]:
feature_rank=pd.DataFrame({'columns':x.columns,
                            'ranking':fit.ranking_,
                            'selected':fit.support_})
feature_rank

,columns,ranking,selected
0,Pregnancies,1,True
1,Glucose,1,True
2,BloodPressure,2,False
3,SkinThickness,4,False
4,Insulin,5,False
5,BMI,1,True
6,DiabetesPedigreeFunction,1,True
7,Age,3,False


In [12]:
recursive_feature_name=feature_rank.loc[feature_rank['selected']==True]
recursive_feature_name

,columns,ranking,selected
0,Pregnancies,1,True
1,Glucose,1,True
5,BMI,1,True
6,DiabetesPedigreeFunction,1,True


In [13]:
x[recursive_feature_name['columns'].values].head()

,Pregnancies,Glucose,BMI,DiabetesPedigreeFunction
0,6,148,33.6,0.627
1,1,85,26.6,0.351
2,8,183,23.3,0.672
3,1,89,28.1,0.167
4,0,137,43.1,2.288


In [14]:
recursive_feature=x[recursive_feature_name['columns'].values]

In [15]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier

In [16]:
feature_selector=SequentialFeatureSelector(RandomForestClassifier(n_estimators=10),
                                           k_features=4,
                                           forward=True,
                                           scoring='accuracy',
                                           cv=4)
feature=feature_selector.fit(np.array(x),y)

In [18]:
forward_elimination_feature_names=list(x.columns[list(feature.k_feature_idx_)])
forward_elimination_feature_names

['Glucose', 'BloodPressure', 'BMI', 'Age']

In [19]:
forward_elimination_feature=x[forward_elimination_feature_names]

In [20]:
feature_selector=SequentialFeatureSelector(RandomForestClassifier(n_estimators=10),
                                           k_features=4,
                                           forward=False,
                                           scoring='accuracy',
                                           cv=4)
feature=feature_selector.fit(np.array(x),y)

In [22]:
backward_elimination_feature_names=list(x.columns[list(feature.k_feature_idx_)])
backward_elimination_feature_names

['Glucose', 'BloodPressure', 'BMI', 'Age']

In [23]:
backward_elimination_feature=x[backward_elimination_feature_names]

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
def build_model(x,y,test_frac):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=test_frac)
    model=LogisticRegression(solver='liblinear').fit(x_train,y_train)
    y_pred=model.predict(x_test)
    
    print("test score:",accuracy_score(y_test,y_pred))

In [26]:
build_model(x,y,0.2)

test score: 0.7662337662337663


In [27]:
build_model(recursive_feature,y,0.2)

test score: 0.8116883116883117


In [28]:
build_model(forward_elimination_feature,y,0.2)

test score: 0.7402597402597403


In [29]:
build_model(backward_elimination_feature,y,0.2)

test score: 0.7987012987012987
